In [85]:
from IPython.display import display
import altair as alt
from altair.utils.data import to_values

def VegaLite(spec):
    bundle = {}
    bundle['application/vnd.vegalite.v4+json'] = spec
    display(bundle, raw=True)

In [86]:
import duckdb

# Le dataset
csv = '../../data/toflit18_all_flows.csv'

# Lire le csv et l'envoyer comme une relation (table)
toflit_flows = duckdb.sql(f"""
SELECT  customs_region,
        CAST(value AS FLOAT) as value,
        year,
        export_import,
        best_guess_region_prodxpart
        best_guess_national_prodxpart,
        best_guess_national_partner,
        best_guess_national_product,
        best_guess_region_prodxpart,
        best_guess_national_region,
        origin_province,
        partner_grouping,
        partner_simplification
FROM read_csv_auto('{csv}', ALL_VARCHAR=TRUE)
""")

In [87]:
# Appliquer les filtres de base (communes aux tous buckets)

base_relation = duckdb.sql(f"""
    FROM toflit_flows
    WHERE customs_region = 'Marseille'
    AND year = '1789'
    AND value IS NOT NULL
    AND export_import = 'Exports'
    AND best_guess_region_prodxpart = '1'
""")

base_relation.show()

┌────────────────┬──────────┬─────────┬───────────────┬───┬─────────────────┬──────────────────┬──────────────────────┐
│ customs_region │  value   │  year   │ export_import │ … │ origin_province │ partner_grouping │ partner_simplifica…  │
│    varchar     │  float   │ varchar │    varchar    │   │     varchar     │     varchar      │       varchar        │
├────────────────┼──────────┼─────────┼───────────────┼───┼─────────────────┼──────────────────┼──────────────────────┤
│ Marseille      │   2312.5 │ 1789    │ Exports       │ … │ Provence        │ France           │ Marseille            │
│ Marseille      │   4237.0 │ 1789    │ Exports       │ … │ NULL            │ France           │ Marseille            │
│ Marseille      │  43636.5 │ 1789    │ Exports       │ … │ Bourgogne       │ France           │ Marseille            │
│ Marseille      │   1593.0 │ 1789    │ Exports       │ … │ Dauphiné        │ France           │ Marseille            │
│ Marseille      │ 194160.0 │ 1789    │ 

In [88]:
# BUCKET 1 (Marseille) - partner_grouping = 'France' AND partner_simplification = 'Marseille'

marseille_bucket = duckdb.sql(f"""
FROM base_relation
WHERE partner_grouping = 'France'
AND partner_simplification = 'Marseille'
""")

marseille_bucket.show()

┌────────────────┬───────────┬─────────┬───────────────┬───┬─────────────────┬──────────────────┬──────────────────────┐
│ customs_region │   value   │  year   │ export_import │ … │ origin_province │ partner_grouping │ partner_simplifica…  │
│    varchar     │   float   │ varchar │    varchar    │   │     varchar     │     varchar      │       varchar        │
├────────────────┼───────────┼─────────┼───────────────┼───┼─────────────────┼──────────────────┼──────────────────────┤
│ Marseille      │    2312.5 │ 1789    │ Exports       │ … │ Provence        │ France           │ Marseille            │
│ Marseille      │    4237.0 │ 1789    │ Exports       │ … │ NULL            │ France           │ Marseille            │
│ Marseille      │   43636.5 │ 1789    │ Exports       │ … │ Bourgogne       │ France           │ Marseille            │
│ Marseille      │    1593.0 │ 1789    │ Exports       │ … │ Dauphiné        │ France           │ Marseille            │
│ Marseille      │    2586.0 │ 1

In [89]:
# BUCKET 2 (France) - partner_grouping = 'France' AND partner_simplification != 'Marseille'

france_bucket = duckdb.sql(f"""
FROM base_relation
WHERE partner_grouping = 'France'
AND partner_simplification != 'Marseille'
""")

france_bucket.show()

┌────────────────┬───────────┬─────────┬───────────────┬───┬─────────────────┬──────────────────┬──────────────────────┐
│ customs_region │   value   │  year   │ export_import │ … │ origin_province │ partner_grouping │ partner_simplifica…  │
│    varchar     │   float   │ varchar │    varchar    │   │     varchar     │     varchar      │       varchar        │
├────────────────┼───────────┼─────────┼───────────────┼───┼─────────────────┼──────────────────┼──────────────────────┤
│ Marseille      │ 26325.771 │ 1789    │ Exports       │ … │ NULL            │ France           │ Dunkerque            │
│ Marseille      │   2198.44 │ 1789    │ Exports       │ … │ NULL            │ France           │ Bayonne              │
│ Marseille      │   3015.46 │ 1789    │ Exports       │ … │ NULL            │ France           │ Dunkerque            │
│ Marseille      │     116.0 │ 1789    │ Exports       │ … │ NULL            │ France           │ Corse                │
│ Marseille      │     154.0 │ 1

In [90]:
# BUCKET 3 (monde) - partner_grouping != 'France'

monde_bucket = duckdb.sql(f"""
FROM base_relation
WHERE partner_grouping != 'France'
""")

# monde_bucket.show()

In [91]:
def group_by_province(relation) -> list[dict]:
    agg = relation.aggregate("""
        origin_province,
        SUM(value) as total_value
    """)

    values = agg.fetchall()
    columns = agg.columns

    return [dict(zip(columns, row)) for row in values]

marseille_bucket_data = group_by_province(marseille_bucket)
france_bucket_data = group_by_province(france_bucket)
monde_bucket_data = group_by_province(monde_bucket)

In [92]:
def visualise(aggregations, title):
    
    VegaLite({
        "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
        "title": title,
        "mark": "bar",
        "data": {
            "values": aggregations
        },
        "encoding": {
            "x": {
                "field": "total_value",
                "type": "quantitative", 
                "title": "value"
            },
            "y": {
                "field": "origin_province",
                "type": "nominal",
                "title": "origin_province",
                "sort": "-x"
            }
        }
    })

In [103]:
visualise(marseille_bucket_data, 'Origine des produits exportés depuis Marseille vers Marseille')

In [104]:
visualise(france_bucket_data, 'Origine des produits exportés depuis Marseille vers partenaires en France')

In [105]:
visualise(monde_bucket_data, "Origine des produits exportés effectués depuis Marseille vers l'étranger")

In [110]:
# Voir qui sont les partenaires de la marchandise 
# exportée depuis Marseille vers la France 
# pour laquelle il n'y a pas d'origin_province

france_bucket = duckdb.sql(f"""
FROM base_relation
WHERE partner_grouping = 'France'
AND partner_simplification != 'Marseille'
""")

def group_by_partner(relation) -> list[dict]:
    agg = relation.aggregate("""
        partner_simplification,
        SUM(value) as total_value
    """)

    values = agg.fetchall()
    columns = agg.columns

    return [dict(zip(columns, row)) for row in values]

france_origin_is_null_grouped_by_partner = group_by_partner(france_bucket)

france_origin_is_null_grouped_by_partner

[{'partner_simplification': 'Dunkerque', 'total_value': 655032.4423675537},
 {'partner_simplification': 'Bayonne', 'total_value': 280171.2106628418},
 {'partner_simplification': 'Corse', 'total_value': 350871.0802078247},
 {'partner_simplification': 'Lorient', 'total_value': 63961.70001220703}]